In [ ]:
#데이터 가져오기
import pickle 

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [ ]:
#데이터 10개 추출
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

In [3]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

c:\Users\HyeonSeong\Hateslop\RAG-1stWeek\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 평가

### Baseline (w/o RAG)

In [ ]:
#GPT 답변 추출
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:56<00:00,  5.68s/it]


In [6]:
predictions

['글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:\n\n1. **경제 성장 둔화**: 글로벌 경제 성장률이 둔화되면서 중앙은행들은 경제를 부양하기 위해 금리를 낮추는 정책을 채택했습니다. 이는 소비와 투자를 촉진하기 위한 조치입니다.\n\n2. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 저축률이 증가하고, 이는 자본 공급을 늘려 금리를 낮추는 요인으로 작용합니다.\n\n3. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 낮추어 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.\n\n4. **글로벌화**: 글로벌화로 인해 자본이 국경을 넘어 자유롭게 이동하면서, 자본 시장의 통합이 이루어졌습니다. 이는 금리의 국제적 동조화를 촉진하고, 전 세계적으로 금리가 낮아지는 현상을 초래했습니다.\n\n5. **중앙은행의 통화 정책**: 2008년 글로벌 금융위기 이후 주요 중앙은행들은 양적 완화(QE)와 같은 비전통적 통화 정책을 통해 대규모로 자산을 매입하고 금리를 낮추는 정책을 시행했습니다. 이는 금융 시장에 유동성을 공급하고 금리를 낮추는 효과를 가져왔습니다.\n\n6. **저물가**: 전 세계적으로 물가 상승률이 낮게 유지되면서 중앙은행들은 금리를 낮게 유지할 수 있는 여유가 생겼습니다. 저물가는 금리 인상의 필요성을 줄이는 요인으로 작용합니다.\n\n이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.',
 '고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높은 위험을 감수하는 투자 상품이나 자산에 집중하는 현상을 말합니다. 이러한 현상은 여러 가지 요인에 의해 발생할 수 있습니다.\n\n1. **저금리 환경**: 중앙은행이 금리를 낮추면, 안전한 자산(예: 국채, 예금)에서 얻을 수 있는 수익률이 감소합니다. 이에 따라 투자자들은 더 높은 수익을 추구하기 위해

In [7]:
#FactualCorrectness: 생성된 답변이 주어진 문맥(또는 참조 데이터)과 비교했을 때 사실적으로 정확한지 평가
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    samples.append(sample)


dataset = EvaluationDataset(samples)

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

#준비된 데이터셋(dataset)과 메트릭(metrics)을 입력으로 받아, 평가 점수를 계산
results = evaluate(dataset, metrics=[factual_correctness_metric])

#평가 결과(results)를 Pandas 데이터프레임으로 변환 후 상위 5개 행 확인
results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:28<00:00,  2.85s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다...   
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높...   
2  디지털화의 진전은 20세기 이후 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행(Investment Banks, IB)들은 다양한...   
4  금융시스템 개혁법은 국가마다 다를 수 있습니다. 예를 들어, 미국의 경우 2010년...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.28  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.21  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.12  
3  2000년대 이후 글로벌 IB들은 

In [8]:
print(results)

{'factual_correctness': 0.1500}


In [9]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [10]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 글로벌 경제 성장률이 둔화되면서 중앙은행들은 경제를 부양하기 위해 금리를 낮추는 정책을 채택했습니다. 이는 소비와 투자를 촉진하기 위한 조치입니다.

2. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 저축률이 증가하고, 이는 자본 공급을 늘려 금리를 낮추는 요인으로 작용합니다.

3. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 낮추어 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.

4. **글로벌화**: 글로벌화로 인해 자본이 국경을 넘어 자유롭게 이동하면서, 자본 시장의 통합이 이루어졌습니다. 이는 금리의 국제적 동조화를 촉진하고, 전 세계적으로 금리가 낮아지는 현상을 초래했습니다.

5. **중앙은행의 통화 정책**: 2008년 글로벌 금융위기 이후 주요 중앙은행들은 양적 완화(QE)와 같은 비전통적 통화 정책을 통해 대규모로 자산을 매입하고 금리를 낮추는 정책을 시행했습니다. 이는 금융 시장에 유동성을 공급하고 금리를 낮추는 효과를 가져왔습니다.

6. **저물가**: 전 세계적으로 물가 상승률이 낮게 유지되면서 중앙은행들은 금리를 낮게 유지할 수 있는 여유가 생겼습니다. 저물가는 금리 인상의 필요성을 줄이는 요인으로 작용합니다.

이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.


In [12]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

#질문과 가장 관련성이 높은 context 추출
def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:56<00:00,  5.63s/it]


In [13]:
#정제된 데이터셋를 가지고 같은 평가 진행
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    samples.append(sample)


dataset = EvaluationDataset(samples)

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

results = evaluate(dataset, metrics=[factual_correctness_metric])

results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:22<00:00,  2.26s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 ...   
1  고수익-고위험 부문으로의 쏠림현상은 저금리 기조 하에서 투자자들이 원하는 수익률을 ...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행들(IB)은 여러 전략을 추진하였습니다. 주요 전략...   
4  금융시스템 개혁법에 대한 정보는 주어진 문맥에서 명확하게 언급되지 않았습니다. 그러...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.67  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.55  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.58  
3  2000년대 이후 글로벌 IB들은 

In [14]:
print(results)

{'factual_correctness': 0.5640}


In [15]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행하였습니다. 이로 인해 글로벌 저금리 현상이 두드러지게 나타났습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여하였습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상이 발생하였습니다.

3. **코로나 팬데믹**: 최근 코로나 사태도 저금리 현상을 지속시키는 요인 중 하나입니다. 팬데믹이 진정되더라도 글로벌 저금리 추세는 계속될 가능성이 높습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.


In [16]:
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
